---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Learning to Rank (LTR) task

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-03-04 01:54:25--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-03-04 01:54:26--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  14.1MB/s    in 1m 44s  

2024-03-04 01:56:10 (13.3 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
wv_embeddings

In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

In [ ]:
# method most_simmilar
def most_similar(word, topn):
    similar_words = wv_embeddings.similar_by_word(word, topn=topn)
    return [word for word, _ in similar_words]

most_sim_words = most_similar('dog', 30)
print(most_sim_words)
print("sim with cat", wv_embeddings.similarity('dog', 'cat'))
print("sim with less similar from the top simiral ", wv_embeddings.similarity('dog', most_sim_words[-1]))
if 'cat' in most_sim_words:
  index = most_sim_words.index('cat')
  print("cat's rank is", index)


['animal', 'dogs', 'mammal', 'cats', 'animals', 'feline', 'bird', 'animal1', 'doggy', 'labrador', 'canine', 'meow', 'cow', 'dog2', 'woof', 'dog1', 'dog3', 'penguin', 'bulldog', 'mammals', 'bark', 'fruit', 'reptile', 'furry', 'carnivore', 'cat', 'horse', 'kitten', 'sheep', 'chihuahua']
sim with cat 0.6852341
sim with less similar from the top simiral  0.6791758
cat's rank is 25


#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

#### Answer 1:
'cat' is not in the top-5 similar words for 'dog', but 'cats" is: ['animal', 'dogs', 'mammal', 'cats', 'animals']
'cat's rank is 25




### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re

# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)



In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    words = tokenizer.tokenize(question.lower())

    question_embed = np.zeros(dim)
    if not words:
      return question_embed

    count = 0
    for word in words:
      if word in embeddings:
        question_embed += embeddings[word]
        count += 1

    if count != 0:
      question_embed/=count

    return question_embed



Теперь у нас есть метод для создания векторного представления любого предложения.

In [ ]:
from nltk.tokenize import WordPunctTokenizer
punct_tokenizer = WordPunctTokenizer()

In [ ]:
embedd = question_to_vec("I love neural networks", wv_embeddings, punct_tokenizer)
embedd[2]

-1.2854121724764507

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

#### Answer 2:
-1.28

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

#### Answer 3:
max(Hits@47 - DCG@1) = 1

Explanation: The maximum value of (Hits@47 - DCG@1) is attained when the model achieves the maximum Hits@47 and the minimum DCG@1. The minimum DCG@1 occurs when the model assigns incorrect ranks for all q_i (rank_q_i_tag > 1), resulting in a value of 0. On the other hand, the maximum Hits@47 is reached when the model ensures rank_q_i_tag <= 47 for all q_i, yielding a value of 1.


<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)


#### Answer 4:

DCG@10 = 1/log_2(1+9) = 1/log_2(10) = log_10(2) = 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = sum(1 for r_q_tag in dup_ranks if r_q_tag<= k)
    return hits_value/len(dup_ranks) if dup_ranks else 0

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = sum(1./np.log2(1+r_q_tag) for r_q_tag in dup_ranks if r_q_tag<= k)
    return dcg_value/len(dup_ranks) if dup_ranks else 0

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!pip install -q gdown
!gdown --id 1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
From (redirected): https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_&confirm=t&uuid=9f94be9c-b59e-4b49-a02e-ceec3914baf0
To: /content/stackoverflow_similar_questions.zip
100% 131M/131M [00:05<00:00, 22.4MB/s]


In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        # strip() to remove leading and trailing whitespaces from each line.
        data.append(line.strip().split("\t"))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')


Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    X = [question_to_vec(question, embeddings, tokenizer, dim)]
    Y = []
    for cand in candidates:
      Y.append(question_to_vec(cand, embeddings, tokenizer, dim))

    cos_sim = cosine_similarity(X, Y)[0]
    sorted_i_cos_sim = sorted(enumerate(cos_sim), key=lambda x: x[1])
    return [(i, candidates[i]) for  i, _ in sorted_i_cos_sim[-1::-1]]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, punct_tokenizer)
        print(ranks)
        print()


[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)



Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?
#### Answer 5:
021


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking1 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, punct_tokenizer)
    # ranks = [(2, c), (0, a), (1, b)]
    # [r[0] for r in ranks] =  [2,0,1]
    # [r[0] for r in ranks].index(0) = 1 meaning the index of value 0 in the array
    # the similar question was first (on index 0) in ex (r[0] = 0)
    # if rank_candidates() are perfectly accurated, the similar question should be first in ranks (on index 0) with rank 0 + 1 = 1
    # we  meassure the accuracy of ranking by checking the similar question's index in ranks
    # in our example [(2, c), (0, a), (1, b)] her index is 1, her rank is 2, when it should've been 1, meaning not accurated!
    wv_ranking1.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking1, k), k, hits_count(wv_ranking1, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.494 | Hits@   5: 0.578
DCG@  10: 0.516 | Hits@  10: 0.644
DCG@ 100: 0.563 | Hits@ 100: 0.875
DCG@ 500: 0.575 | Hits@ 500: 0.973
DCG@1000: 0.578 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# Get NLTK stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def preprocessSpacy(question):
    question = question.lower()

    # Use spacy for tokenization
    doc = nlp(question)
    words = [token.text for token in doc]

    return [word for word in words if word not in stop_words]

In [ ]:
def preprocess(question, tokenizer):
    question = question.lower()
    return [word for word in tokenizer.tokenize(question) if word not in stop_words]

In [ ]:
def applyPreprocess(tokenizer, question, question_tag):
    return preprocess(question, tokenizer) + preprocess(question_tag, tokenizer)

In [ ]:
def generate_words(tokenizer):
  words = []
  for line in train_data:
    q, *q_tag = line
    words.append(applyPreprocess(tokenizer, q, q_tag[0]))
  return words

In [ ]:
from nltk.tokenize import RegexpTokenizer
regexp_tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
words = generate_words(regexp_tokenizer)
print(words[:10])


[['converting', 'string', 'list', 'convert', 'google', 'results', 'object', 'pure', 'js', 'python', 'object'], ['html', '5', 'canvas', 'javascript', 'use', 'making', 'interactive', 'drawing', 'tool', 'event', 'handling', 'geometries', 'three', 'js'], ['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'list', 'items', 'unordered', 'list', 'php'], ['insert', 'cookiecollection', 'cookiecontainer', 'c', 'create', 'cookie', 'string', 'send'], ['updating', 'one', 'element', 'bound', 'observable', 'collection', 'wpf', 'update', 'changes', 'list', 'item', 'list'], ['mongodb', 'error', 'find', 'retrieve', 'queried', 'element', 'object', 'array', 'mongodb', 'collection'], ['select2', 'displaying', 'search', 'results', 'use', 'jquery', 'ajax', 'outside', 'domain'], ['using', 'reduce', 'merge', 'multiple', 'data', 'frames', 'passing', 'arguments', 'without', 'defining', 'function', 'outside', 'reduce', 'syntax', 'r', 'merge', 'list', 'data', 'frames', 'one', 'data', 'frame', 'missing', 'value

In [ ]:
for line in train_data[:3]:
    q, q_tag = line
    print(q)
    print(q_tag)
    print()


converting string to list
Convert Google results object (pure js) to Python object

Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?
Event handling for geometries in Three.js?

Sending array via Ajax fails
Getting all list items of an unordered list in PHP



In [ ]:
for line in validation_data[:5]:
    q, *examples = line
    print(q)
    print(examples[0])
    print()
    for r in examples[1:10]:
      print(r)
    print()
    break

How to print a binary heap tree without recursion?
How do you best convert a recursive function to an iterative one?

How can i use ng-model with directive in angular js
flash: drawing and erasing
toggle react component using hide show classname
Use a usercontrol from another project to current webpage
~ Paths resolved differently after upgrading to ASP.NET 4
Materialize datepicker - Rendering when an icon is clicked
Creating PyPi package - Could not find a version that satisfies the requirement iso8601
How can I analyze a confusion matrix?
How do I declare a C array in Swift?



In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=2,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
# Print similarity
similarity = embeddings_trained.similarity('converting', 'convert')
print(f"Similarity between 'converting' and 'convert': {similarity}")

Similarity between 'converting' and 'convert': 0.8542309403419495


In [ ]:
regexp_tokenizer

RegexpTokenizer(pattern='\\w+', gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)

In [ ]:
wv_ranking2 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    # print("q", q)
    # if ex:
    #   print("ex", ex[0])
    ranks = rank_candidates(q, ex, embeddings_trained, regexp_tokenizer)
    wv_ranking2.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking2, k), k, hits_count(wv_ranking2, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.498 | Hits@   5: 0.586
DCG@  10: 0.521 | Hits@  10: 0.657
DCG@ 100: 0.567 | Hits@ 100: 0.877
DCG@ 500: 0.580 | Hits@ 500: 0.979
DCG@1000: 0.582 | Hits@1000: 1.000


In [ ]:
from nltk.tokenize import TreebankWordTokenizer
treebank_tokenizer = TreebankWordTokenizer()
words2 = generate_words(treebank_tokenizer)
print(words2[:10])

[['converting', 'string', 'list', 'convert', 'google', 'results', 'object', '(', 'pure', 'js', ')', 'python', 'object'], ['html', '5', 'canvas', 'javascript', 'use', 'making', 'interactive', 'drawing', 'tool', '?', 'event', 'handling', 'geometries', 'three.js', '?'], ['sending', 'array', 'via', 'ajax', 'fails', 'getting', 'list', 'items', 'unordered', 'list', 'php'], ['insert', 'cookiecollection', 'cookiecontainer', '?', 'c', '#', 'create', 'cookie', 'string', 'send'], ['updating', 'one', 'element', 'bound', 'observable', 'collection', 'wpf-', 'update', 'changes', 'list', 'item', 'list'], ['mongodb', 'error', 'find', '(', ')', 'retrieve', 'queried', 'element', 'object', 'array', 'mongodb', 'collection'], ['select2', 'displaying', 'search', 'results', 'use', 'jquery', 'ajax', 'outside', 'domain', '?'], ['using', 'reduce', 'merge', 'multiple', 'data', 'frames', 'passing', 'arguments', 'without', 'defining', 'function', 'outside', 'reduce', '(', 'syntax', ')', 'r', '-', 'merge', 'list', '

In [ ]:
embeddings_trained2 = Word2Vec(words2, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=2,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
# Print similarity
similarity = embeddings_trained2.similarity('converting', 'convert')
print(f"Similarity between 'converting' and 'convert': {similarity}")

Similarity between 'converting' and 'convert': 0.8723545670509338


In [ ]:
wv_ranking3 = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained2, treebank_tokenizer)
    wv_ranking3.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking3, k), k, hits_count(wv_ranking3, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.351 | Hits@   1: 0.351
DCG@   5: 0.448 | Hits@   5: 0.528
DCG@  10: 0.476 | Hits@  10: 0.615
DCG@ 100: 0.521 | Hits@ 100: 0.837
DCG@ 500: 0.539 | Hits@ 500: 0.968
DCG@1000: 0.542 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
I tried several tokenazers and found RegexpTokenizer just slightly better.

Normalization makes vector representations more consistent and reduce the vocabulary size. I added lowercasing and filtered by stop words.

In this homework I tried only Word2Vec to generate embeddings.

I don't have enough experience with this task to make some educated guesses, but I would assume that the performance is not satisfactory due to the limited number of words relevant to the task. I utilized words from two sentences, which enhanced the accuracy compared to using only one sentence. If we had a longer text with more sentences per theme, it could potentially lead to improved accuracy.

